### Custom Models and Training with TensorFlow

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

In [2]:
def huber_fn(preds, labels):
    error = labels - preds
    is_small = tf.abs(error)<1
    squared_loss = tf.square(error)/2
    linear_loss = tf.abs(error)-0.5
    return tf.where(is_small, squared_loss, linear_loss)

In [31]:
(X_train,y_train),(X_test,y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train.shape

(60000, 28, 28)

In [32]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape = [28,28]))
model.add(keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"))
model.add(keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"))
model.add(keras.layers.Dense(10, activation="softmax", kernel_initializer="he_normal"))
model.compile(optimizer="nadam", loss = "sparse_categorical_crossentropy",metrics=["accuracy"])
history = model.fit(X_train,y_train, epochs = 10)

Epoch 1/10
1875/1875 [==============================] - 4s 2ms/step - loss: 3.5379 - accuracy: 0.7274
Epoch 2/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.7684 - accuracy: 0.7740
Epoch 3/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.6215 - accuracy: 0.8009
Epoch 4/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.5742 - accuracy: 0.8123
Epoch 5/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.5209 - accuracy: 0.8206
Epoch 6/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.4996 - accuracy: 0.8257
Epoch 7/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.4908 - accuracy: 0.8295
Epoch 8/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.4592 - accuracy: 0.8406
Epoch 9/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.4495 - accuracy: 0.8435
Epoch 10/10
1875/1875 [==============================] - 3s 1ms/step - loss: 0.436

In [5]:
model.evaluate(X_test,y_test)

313/313 [==============================] - 0s 796us/step - loss: 0.1411 - accuracy: 0.9633


[0.14109180867671967, 0.9632999897003174]

In [6]:
class MyDense(keras.layers.Layer):

    def __init__(self, units, activation, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = activation

    def build(self, input_shape):
        self.kernel = self.add_weight(name="kernel", shape = [input_shape[:-1],self.units], initializer= "glorot_normal")
        self.bias = self.add_weight(name="bias", shape = [self.units],initializer="zeros")
        super().build(input_shape)
    
    def call(self,X):
        return self.activation(X @ self.kernel + self.bias)
    
    def compute_output_shape(self, input_shape):
       return tf.TensorShape(input_shape.as_list()[:-1] + [self.units])

    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            "units":self.units,
            "activation": keras.activations.serialize(self.activation)
        }

In [7]:
class MyMultiLayer(keras.layers.Layer):
    def call(self,x):
        x1,x2 = x
        return [x1 + x2, x1 * x2, x1/x2]
    def compute_output_shape(self, batch_input_shape):
        b1,b2 = batch_input_shape
        return [b1,b1,b1]
    


### Custom Models

In [8]:
class ResidualLayer(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(n_neurons, activation='elu', kernel_initializer='he_normal') for _ in range(n_layers)]
    
    def call(self,inputs):
        z = inputs
        for layer in self.hidden:
            z = layer(z)
        return z + inputs

class ResidualModel(keras.Model):
    def __init__(self,output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(30, activation = "elu", kernel_initializer= "he_normal")
        self.b1 = ResidualLayer(2, 30)
        self.b2 = ResidualLayer(2, 30)
        self.out = keras.layers.Dense(output_dim)
    
    def call(self,inputs):
        z1 = self.hidden1(inputs)
        for _ in range(1+3):
            z1 = self.b1(z1)
        z1 = self.b2(z1)
        return self.out(z1)

In [9]:
(X_train_full,y_train_full),(X_test,y_test) = keras.datasets.mnist.load_data()

In [10]:
X_train = X_train_full[:50000]
y_train = y_train_full[:50000]
X_val = X_train_full[50000:]
y_val = y_train_full[50000:]

### Losses and metrics based on model Internals

In [11]:
class ReconstructingRegressor(keras.models.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(30, activation="selu",
                                          kernel_initializer="lecun_normal")
                       for _ in range(5)]
        self.out = keras.layers.Dense(output_dim)
        self.reconstruct = keras.layers.Dense(8)
        self.reconstruction_mean = keras.metrics.Mean(name="reconstruction_error")


    def call(self, inputs, training=None):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        recon_loss = tf.reduce_mean(tf.square(reconstruction - inputs))
        self.add_loss(0.05 * recon_loss)
        if training:
            result = self.reconstruction_mean(recon_loss)
            self.add_metric(result)
        return self.out(Z)

In [12]:
l2_reg = keras.regularizers.l2(0.05)

model = keras.models.Sequential([
    keras.layers.Dense(30, activation='elu', kernel_initializer="he_normal", kernel_regularizer=l2_reg),
    keras.layers.Dense(1,kernel_regularizer=l2_reg)
])

def random_batch(X,y, batch_size=32):
    idx =np.random.randint(len(X), size = batch_size)
    return X[idx], y[idx]

def print_status_bar(iteration, total, loss, metrics = None):
    metrics = " - ".join(["{}: {:.4f}".format(m.name, m.result()) for m in [loss] + (metrics or [])])
    end = "" if iteration > total else "\n"
    print("\r{}/{} - ".format(iteration, total) + metrics, end=end)

In [13]:
n_epochs = 5
batch_size= 32
n_steps = len(X_train)//batch_size
optimizer = keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = keras.losses.mean_squared_error
mean_loss=keras.metrics.Mean()
metrics= [keras.metrics.MeanAbsoluteError()]

In [14]:
for epoch in range(1, n_epochs+1):
    print("Epoch {}/{}".format(epoch, n_epochs))
    for step in range(1, n_steps+1):
        X_batch,y_batch = random_batch(X_train, y_train)
        with tf.GradientTape() as tape:
            y_pred = model(X_batch, training = True)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        mean_loss(loss)
        for metric in metrics:
            metric(y_batch, y_pred)
        print_status_bar(step * batch_size, len(y_train), mean_loss, metrics)
    print_status_bar(len(y_train), len(y_train), mean_loss, metrics)
    for metric in [mean_loss] + metrics:
        metric.reset_states       

Epoch 1/5
32/50000 - mean: 5689.9185 - mean_absolute_error: 50.6388
64/50000 - mean: 4697.6182 - mean_absolute_error: 46.0034
96/50000 - mean: 4009.8201 - mean_absolute_error: 42.3720
128/50000 - mean: 3599.4875 - mean_absolute_error: 39.6180
160/50000 - mean: 3225.8879 - mean_absolute_error: 37.2500
192/50000 - mean: 2985.6814 - mean_absolute_error: 35.5969
224/50000 - mean: 2780.7273 - mean_absolute_error: 34.3114
256/50000 - mean: 2592.0376 - mean_absolute_error: 33.0310
288/50000 - mean: 2418.4866 - mean_absolute_error: 31.8033
320/50000 - mean: 2275.7156 - mean_absolute_error: 30.7451
352/50000 - mean: 2149.6729 - mean_absolute_error: 29.8247
384/50000 - mean: 2031.8998 - mean_absolute_error: 28.8912
416/50000 - mean: 1930.8745 - mean_absolute_error: 28.1505
448/50000 - mean: 1855.2179 - mean_absolute_error: 27.5888
480/50000 - mean: 1796.1243 - mean_absolute_error: 27.2254
512/50000 - mean: 1812.9812 - mean_absolute_error: 27.5427
544/50000 - mean: 1878.7560 - mean_absolute_error

In [15]:
def cube(x):
    return x**3
tf_cube = tf.function(cube)

@tf.function
def cube(x):
    return x**3


### Exercises
1. API to create computational graphs that facilitate deep learning functionality. Other libraries include the higher level wrapper keras, and pytorch
2. Not completely, numpy functions arent traced and dont automatically add nodes to a graph.
3. One is 32 bit the other is 64 bit, tensorflow defaults to 32 bit integers.
4. 
5. Custom losses for custom models
6. Depends on what you want to track, accuracy, validation loss, etc.
7. Custom layer is good for a layer that you will use multiple times, custom model is good for the architecture of the whole model itself, or if you plan to treat it as a separate model with a final output.
8. When you want to monitor special aspects of the model that arent accessible with .fit
9. Yes, keras will automatically use autograph
10. 
11. When you need it, computation.

In [19]:
class LayerNorm(keras.layers.Layer):
    def __init__(self):
        super().__init__()
    
    def build(self, input_shape):
        self.a = self.add_weight(name="a", shape = input_shape[-1:],initializer="ones")
        self.b = self.add_weight(name="b", shape = input_shape[-1:],initializer="zeros")
        super().build(input_shape)

    def call(self, x):
        mean, variance = tf.nn.moments(x, axes=-1, keepdims=True)
        return self.a * ((x-mean)/(tf.math.sqrt(variance) + 0.0001)) + self.b     

    def compute_output_shape(sefl, input_shape):
        return input_shape

    def get_config(self):
        base_config = super().get_config()   
        return {**base_config}

In [21]:
X = X_train.astype(np.float32)

custom_layer_norm = LayerNorm()
keras_layer_norm = keras.layers.LayerNormalization()

tf.reduce_mean(keras.losses.mean_absolute_error(
    keras_layer_norm(X), custom_layer_norm(X)))

random_alpha = np.random.rand(X.shape[-1])
random_beta = np.random.rand(X.shape[-1])

custom_layer_norm.set_weights([random_alpha, random_beta])
keras_layer_norm.set_weights([random_alpha, random_beta])

tf.reduce_mean(keras.losses.mean_absolute_error(
    keras_layer_norm(X), custom_layer_norm(X)))

<tf.Tensor: shape=(), dtype=float32, numpy=3.0679976e-07>

In [45]:

(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full.astype(np.float32) / 255.
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test.astype(np.float32) / 255.

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax"),
])

n_epochs = 5
batch_size = 32
n_steps = len(X_train) // batch_size
optimizer = keras.optimizers.Nadam(lr=0.01)
loss_fn = keras.losses.sparse_categorical_crossentropy
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.SparseCategoricalAccuracy()]

C:\Users\carlos\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [48]:
from collections import OrderedDict
from tqdm import trange

with trange(1, n_epochs + 1, desc="All epochs") as epochs:
    for epoch in epochs:
        with trange(1, n_steps + 1, desc="Epoch {}/{}".format(epoch, n_epochs)) as steps:
            for step in steps:
                X_batch, y_batch = random_batch(X_train, y_train)
                with tf.GradientTape() as tape:
                    y_pred = model(X_batch)
                    main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
                    loss = tf.add_n([main_loss] + model.losses)
                gradients = tape.gradient(loss, model.trainable_variables)
                optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                for variable in model.variables:
                    if variable.constraint is not None:
                        variable.assign(variable.constraint(variable))                    
                status = OrderedDict()
                mean_loss(loss)
                status["loss"] = mean_loss.result().numpy()
                for metric in metrics:
                    metric(y_batch, y_pred)
                    status[metric.name] = metric.result().numpy()
                steps.set_postfix(status)
            y_pred = model(X_valid)
            status["val_loss"] = np.mean(loss_fn(y_valid, y_pred))
            status["val_accuracy"] = np.mean(keras.metrics.sparse_categorical_accuracy(
                tf.constant(y_valid, dtype=np.float32), y_pred))
            steps.set_postfix(status)
        for metric in [mean_loss] + metrics:
            metric.reset_states()

All epochs: 100%|██████████| 5/5 [02:10<00:00, 26.10s/it]
